# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [5]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Firefox()
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()
time.sleep(3)
html_source = browser.page_source
browser.close()
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [24]:
values=[]
i=0
for tr in table.find_all('tr'):
    if i==0:
        i=1
        continue
    elif tr.findAll('td')[0].getText()=='SO2':
        if i%8==1:
            val=tr.findAll('td')[3].getText()
            i+=1
    elif i>8:
        break
    else:
        val=tr.findAll('td')[1].getText()
        i+=1
    values.append(val)
print(values)

['1.80', '1.90', '2.20', '2.30', '3.10', '2.70', '2.20', '2.40']


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [36]:
values=[]
i=0
for tr in table.find_all('tr'):
    if i==0:
        i=1
        continue
    elif tr.findAll('td')[0].getText() in['SO2','CO']:
        i=1
    elif i%8==1:
        val=tr.findAll('td')[3].getText()
        i+=1
    else:
        val=tr.findAll('td')[1].getText()
        i+=1
    print(tr.findAll('td')[0])
    print(val)
    values.append(val)

<td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td>
0.34


IndexError: list index out of range

In [34]:
values

['1.80', '0.34']

In [6]:
table

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="12" valign="top">ppb</td><td>2018/01</td><td>1.80</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/02</td><td>1.90</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/03</td><td>2.20</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/04</td><td>2.30</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/05</td><td>3.10</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/06</td><td>2.70</td><td> </td>
</tr><tr style="color:Black;">
<td>

In [22]:
values=[]
i=0
for tr in table.find_all('tr'):
    if i==0:
        i=1
        continue
    elif tr.findAll('td')[0].getText()=='SO2':
        if i%8==1:
            val=tr.findAll('td')[3].getText()
            i+=1
    elif i>8:
        break
    else:
        val=tr.findAll('td')[1].getText()
        i+=1
    values.append(val)

In [23]:
values

['1.80', '1.90', '2.20', '2.30', '3.10', '2.70', '2.20', '2.40', '2.10']

In [19]:
table.find_all('tr')[2].findAll('td')

[<td>2018/02</td>, <td>1.90</td>, <td> </td>]

In [16]:
dir(a)

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 'find',
 'findAl

In [24]:
a.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

AttributeError: 'NoneType' object has no attribute 'next_sibling'